**Preprocessing**

In [ ]:
import pandas as pd
import re

# ========================================
# Helper Function: Remove Emojis from Column Names
# ========================================
def remove_emojis_from_columns(df):
    """Remove emojis and non-alphanumeric characters from DataFrame column names."""
    df.columns = [re.sub(r'[^\w\s]', '', col) for col in df.columns]
    return df

# ========================================
# Load Data
# ========================================
# Load CSV files into DataFrames
deal_df = pd.read_csv('/content/Deals.csv')
founders_df = pd.read_csv('/content/Founders.csv')
startup_df = pd.read_csv('/content/Startups.csv')

# ========================================
# Clean Column Names
# ========================================
# Remove emojis from column names
deal_df = remove_emojis_from_columns(deal_df)
founders_df = remove_emojis_from_columns(founders_df)
startup_df = remove_emojis_from_columns(startup_df)

# ========================================
# Define Columns to Drop
# ========================================
deal_columns_to_drop = [
    'Email for CC', 'Last Interaction', 'Track for Email',
    'Team Main Email', 'Record ID', 'Notes', 'Pitch Deck', 'Startup Info',
    'Cactus Owner', 'Cactus Team', 'DDIC planning', 'IC Date', 'Attachments',
    'Meetings  KOLs', 'Founders Info', '10K Kickoff', 'Reason for rejection', 'Feedback Sending',
    'Days since last interaction', 'Interaction Status', 'Cactus Owner Archive',
    'Date Created', 'Interaction', 'Interaction Filter', 'DD Loom', 'Interaction email',
    'Startup copy', 'Emails', 'Analyst from Cactus Owner', 'OE', 'Investment track for links',
    'Startups Feedback', 'Presented at weekly', 'Committee Status', 'OnlineCourse', 'Second Time Email Status',
    'TImeOs', 'TImeOs 2'
]

founders_columns_to_drop = [
    'Email', 'LinkedIn', 'Phone', 'Picture', 'email lower',
    'Record ID', 'Notes', 'Path For Email', 'Name', 'Title', 'Created Date', 'Startup Deal',
    '10K Kickoff', 'First Name', 'Last Name', 'Automation helper Is name correct', 'Link to founder update form',
    'email_lower', ' Startup'
]

startup_columns_to_drop = [
    'Email(from_Founders)', 'Record_ID', 'Notes',
    'Team Main Email', 'Path For Email', 'Founders', 'Cactus Owner from Deal', 'Website', 'Elevator Pitch', 'What problem you are solving',
    'How are you solving the problem', '50 BGU', 'Pitch Deck', 'Refferance', 'Deal', 'KOL  Potenital Clients',
    'ToDo  Tasks ', 'Autonumber', 'Pitch', 'Meetings', 'Pre Application', '10K Tutorial email sent', 'Email from Founders', 'date Created',
    'Inside Committee 10K Dec 21', 'link to cohort', '10K Tutorial Email Status', 'Email attachments ', 'AgreementMentorship', 'Startup Team Main Email',
    'IP Check Documents', '10K Kickoff', 'Cactus Team', 'Number of founders', 'Record ID', 'Field 41', 'Faculty from  Founders', ' Founders'
]

# ========================================
# Modify Specific Columns
# ========================================
# Modify `Path` in Deal table
if "Path " in deal_df.columns:
    deal_df = deal_df[~deal_df["Path "].isin([
        "Get Ahead 20,000$ (Legacy)", "Get Started 5,000$ (Legacy)", "Give Back 5,000$ (Legacy)", "The IMPACT"
    ])]
    deal_df.loc[:, "Path "] = deal_df["Path "].replace({
        "THE LAUNCHER - 100,000 ₪ (Legacy)": "The LAUNCHER",
        "THE KICKSTARTER - 10,000₪ (Legacy)": "The KICKSTARTER"
    })

# Modify `Path` in Startup table
if "Path" in startup_df.columns:
    startup_df = startup_df[~startup_df["Path"].isin([
        "Get Ahead 20,000$ (Legacy)", "Get Started 5,000$ (Legacy)", "Give Back 5,000$ (Legacy)", "The IMPACT"
    ])]
    startup_df.loc[:, "Path"] = startup_df["Path"].replace({
        "THE LAUNCHER - 100,000 ₪ (Legacy)": "The LAUNCHER",
        "THE KICKSTARTER - 10,000₪ (Legacy)": "The KICKSTARTER"
    })

# Function to clean up trailing or leading commas in the Industry column
def clean_industry_values(industry_value):
    if isinstance(industry_value, str):  # Ensure the value is a string before processing
        return industry_value.strip().strip(',')
    return industry_value

# Apply the cleaning function to the Industry column
startup_df['Industry'] = startup_df['Industry'].apply(clean_industry_values)

# ========================================
# Clean Deal Table
# ========================================
# Drop unnecessary columns
cleaned_deal_df = deal_df.drop(columns=[col for col in deal_columns_to_drop if col in deal_df.columns], errors='ignore')

# Fill missing values in 'Cohort'
if "Cohort" in cleaned_deal_df.columns:
    cleaned_deal_df["Cohort"] = cleaned_deal_df["Cohort"].fillna("6th")

# Rename column for consistency
cleaned_deal_df.rename(columns={"Path ": "Path"}, inplace=True)

# Add 'Year' column without modifying 'Cohort'
cleaned_deal_df["Cohort_Copy"] = cleaned_deal_df["Cohort"].str.replace("th", "", regex=True)
cleaned_deal_df = cleaned_deal_df[pd.to_numeric(cleaned_deal_df["Cohort_Copy"], errors="coerce").notna()]
cleaned_deal_df["Cohort_Copy"] = cleaned_deal_df["Cohort_Copy"].astype(int)
cleaned_deal_df["Year"] = 2020 + ((cleaned_deal_df["Cohort_Copy"] - 3) // 2)

# ========================================
# Clean Founders Table
# ========================================
# Drop unnecessary columns
cleaned_founders_df = founders_df.drop(columns=[col for col in founders_columns_to_drop if col in founders_df.columns], errors='ignore')

# Standardize 'Faculty' column
if "Faculty" in cleaned_founders_df.columns:
    cleaned_founders_df["Faculty"] = cleaned_founders_df["Faculty"].str.strip()
    faculty_corrections = {
        "Engnerring": "Engineering", "Engnerring ": "Engineering", "Engineering ": "Engineering", "engineering": "Engineering",
        "Business & Management": "Business and Management", "business and management": "Business and Management",
        "Natural sciences": "Natural Sciences", "natural sciences": "Natural Sciences",
        "Humanities & Social Sciences": "Humanities and Social Sciences", "humanities and social sciences": "Humanities and Social Sciences"
    }
    cleaned_founders_df["Faculty"] = cleaned_founders_df["Faculty"].replace(faculty_corrections)

# Handle 'Cohort' and add 'Year'
if "Cohort" in cleaned_founders_df.columns:
    cleaned_founders_df["Cohort_Copy"] = cleaned_founders_df["Cohort"].apply(lambda x: None if x in ["Cactus Academy", "Checks"] else int(re.search(r'\d+', str(x)).group()) if re.search(r'\d+', str(x)) else None)
    start_year = 2019
    cleaned_founders_df["Year"] = cleaned_founders_df["Cohort_Copy"].apply(lambda x: start_year + ((x - 1) // 2) if pd.notna(x) else None)
    cleaned_founders_df["Year"] = cleaned_founders_df["Year"].astype("Int64")

cleaned_founders_df.loc[
    cleaned_founders_df["Faculty"].str.strip().str.lower().fillna("").isin(["cognitive science", "cognitive sciences"]),
    "Faculty"
] = "Cognitive Science"
# ========================================
# Clean Startup Table
# ========================================
# Drop unnecessary columns
cleaned_startup_df = startup_df.drop(columns=[col for col in startup_columns_to_drop if col in startup_df.columns], errors='ignore')

# Fill missing values in 'MasterPhD'
if "MasterPhD " in cleaned_startup_df.columns:
    cleaned_startup_df["MasterPhD "] = cleaned_startup_df["MasterPhD "].fillna("Undergraduate")

# Drop rows with missing values
cleaned_startup_df = cleaned_startup_df.dropna()

# ========================================
# Save Cleaned DataFrames to CSV
# ========================================
cleaned_deal_df.to_csv('Cleaned_Deal_Data.csv', index=False)
cleaned_founders_df.to_csv('Cleaned_Founders_Data.csv', index=False)
cleaned_startup_df.to_csv('Cleaned_Startup_Data.csv', index=False)

print("Data cleaning complete. Cleaned files saved.")


Data cleaning complete. Cleaned files saved.
